In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random

from keras.layers import Dense, InputLayer,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
import pickle
import keras


In [ ]:
# Initializing the category of classification
Categories = ["NORMAL", "PNEUMONIA"]

## Preparing Trainig Data

In [ ]:
DataDir_Tranining = "chest_xray/train"

# Converting the image to Grayscale
for category in Categories:
    path = os.path.join(DataDir_Tranining, category)
    for img in os.listdir(path):
        imgs = os.path.join(path, img)
        img_array = cv2.imread(imgs, cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
img_size = 100 # Initializing the size of image

In [ ]:
# Image after resizing and grayscale conversion
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(new_array, cmap="gray")
plt.show()


In [ ]:
training_data = []

# Defining function for training data according to the categories converting the images into gray scale with resolution
# Converting the images into array

def create_train_data():
    for category in Categories:
        path = os.path.join(DataDir_Tranining, category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass


create_train_data()
print(len(training_data))


In [ ]:
random.shuffle(training_data)

In [ ]:
X_train = []
y_train = []

# Splitting the features and labels
for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

In [ ]:
X_train = np.array(X_train).reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

## Preparing Validation Data

In [ ]:
# We doing doing same with training data
validation_data = []
DataDir_Validation = "chest_xray/validation"

def create_validation_data():
    for category in Categories:
        path = os.path.join(DataDir_Validation, category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                validation_data.append([new_array, class_num])
            except Exception as e:
                pass

create_validation_data()
print(len(validation_data))

In [ ]:
random.shuffle(validation_data)

In [ ]:
X_val = []
y_val = []

for features, label in validation_data:
    X_val.append(features)
    y_val.append(label)


In [ ]:

X_val = np.array(X_val).reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

## Preparing Testing Data

In [ ]:
testing_data = []
DataDir_Testing = "chest_xray/test"
def create_test_data():
    for category in Categories:
        path = os.path.join(DataDir_Testing, category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                testing_data.append([new_array, class_num])
            except Exception as e:
                pass

create_test_data()
print(len(testing_data))


In [ ]:
random.shuffle(testing_data)

In [ ]:
X_test = []
y_test = []

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)

In [ ]:
X_test = np.array(X_test).reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

## Model Building

In [ ]:
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0


In [ ]:

def Pneumnia_Classifier(input_shape):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(.2))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model



In [ ]:
BATCH_SIZES = [2, 4, 16, 32, 64]
EPOCHS_LIST = [10, 20, 30]

results = []
best_acc = -1
best_training_acc = -1
best_validation_acc = -1
best_info = None
input_size =X_train.shape[1:]

In [ ]:
for ep in EPOCHS_LIST:
    for bs in BATCH_SIZES:
        print(f"Starting run: batch_size={bs}, epochs={ep}")
        tf.keras.backend.clear_session()
        m = Pneumnia_Classifier(input_size)
        history = m.fit(X_train, y_train, batch_size=bs,epochs=ep, validation_data=(X_val, y_val), verbose=1)
        acc_mean = round(np.mean(history.history['accuracy']) * 100, 2)
        acc_val_mean = round(np.mean(history.history['val_accuracy']) * 100, 2)
        score = m.evaluate(X_test, y_test, verbose=0)
        curr_acc = round(score[1] * 100, 2)
        results.append((bs, ep, acc_mean, acc_val_mean, curr_acc))
        if curr_acc > best_acc:
            best_acc = curr_acc
            best_training_acc = acc_mean
            best_validation_acc = acc_val_mean
            best_info = f'Batch_Size : {bs}, Epochs : {ep}, Training_Accuracy : {best_training_acc}, Validation_Accuracy : {best_validation_acc}, Test_Accuracy : {best_acc}'

In [ ]:
print("Best Result -> ", best_info)

In [ ]:
df = pd.DataFrame(results, columns=['Batch_Size', 'Epochs', 'Training_Accuracy', 'Validation_Accuracy', 'Test_Accuracy'])
df.head()

In [ ]:
m.summary()

In [ ]:
model_name = f'Model/Pneumonia_Detection_CNN_{round(best_acc, 0)}.keras'
tf.keras.models.save_model(m, model_name)